# week-18 class

1. Look up the Adam optimization functions in PyTorch
https://pytorch.org/docs/stable/optim.html . How does it work? Try at least one other
optimization function with the diabetes dataset shown in class. How does the model
perform with the new optimizer? Did it perform better or worse than Adam? Why do you
think that is?


In [140]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.metrics import classification_report, plot_confusion_matrix
import pydotplus


from imblearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours
from IPython.display import Image

In [252]:
diabetes_df = pd.read_csv("A:\launch_code_STL\Final_Homework\week-13\diabetes.csv")
diabetes_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [253]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = diabetes_df.drop('Outcome', axis=1).values
y = diabetes_df['Outcome'].values

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42, stratify=y)

# #Standardize the data
sc= StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [254]:
import torch.nn as nn
import torch
import torch.nn.functional as F # Activation function

#create tensors 
X_train = torch.FloatTensor(X_train) 
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

print(X_train)

tensor([[ 0.9314,  2.0179,  0.7807,  ...,  0.4315, -0.3748,  0.6321],
        [ 0.6326, -1.1486,  0.4654,  ..., -0.1198, -0.2942,  0.7170],
        [-0.5625, -0.4769, -0.2703,  ..., -0.2096,  2.7452,  0.0381],
        ...,
        [-0.8613, -0.7648,  0.0450,  ...,  0.7648, -0.7838, -0.3014],
        [ 0.6326,  2.2099,  1.2010,  ...,  0.4315, -0.6047,  2.7537],
        [ 0.0351,  0.7385, -0.5856,  ..., -0.3378, -0.5778,  0.2927]])


In [255]:
#artificial neural network
class ANN_Model(nn.Module):
    def __init__(self, input_features=8,hidden1=20,hidden2=20,out_features=2):
        super().__init__() #super is a computed indirect reference. So, it isolates changes
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
        
    def forward(self, x):
        #apply activation functions
        x = F.relu(self.layer_1_connection(x))
        x = F.relu(self.layer_2_connection(x))
        x = self.out(x)
        return x

In [256]:
torch.manual_seed(42)

#create instance of model
ann = ANN_Model()

In [257]:
#loss function
loss_function = nn.CrossEntropyLoss()

#optimizer
optimizer = torch.optim.SGD(ann.parameters(), lr=0.01, momentum=0.7)
# optimizer = torch.optim.Adam(ann.parameters(),lr=0.01)
# optimizer = torch.optim.RMSprop(ann.parameters(), lr=0.02) 
# optimizer = torch.optim.Adadelta(ann.parameters(), lr=1.0, rho=0.9, eps=1e-06, weight_decay=0)
# optimizer = torch.optim.Adagrad(ann.parameters(), lr=0.01)
# optimizer =torch.optim.AdamW(ann.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)
# optimizer = torch.optim.SparseAdam(ann.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08)
# optimizer = torch.optim.Adamax(ann.parameters(), lr=0.005)
# optimizer = torch.optim.NAdam(ann.parameters(), lr=0.002, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, momentum_decay=0.004)
# optimizer = torch.optim.RAdam(ann.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
# optimizer = torch.optim.Rprop(ann.parameters(), lr=0.01, etas=(0.5, 1.2), step_sizes=(1e-06, 50))


In [258]:
#run model through multiple epochs/iterations
final_loss = []
n_epochs = 500
for epoch in range(n_epochs):
    y_pred = ann.forward(X_train)
    loss = loss_function(y_pred, y_train)
    final_loss.append(loss)
    
    if epoch % 10 == 1:
        print(f'Epoch number: {epoch} with loss: {loss}')
        
    optimizer.zero_grad() #zero the gradient before running backwards propagation
    loss.backward() 
    optimizer.step() #perform one optimization step each epoch

Epoch number: 1 with loss: 0.6608980894088745
Epoch number: 11 with loss: 0.6561811566352844
Epoch number: 21 with loss: 0.6515836119651794
Epoch number: 31 with loss: 0.6476818323135376
Epoch number: 41 with loss: 0.6441858410835266
Epoch number: 51 with loss: 0.6408833265304565
Epoch number: 61 with loss: 0.6376416087150574
Epoch number: 71 with loss: 0.6343832612037659
Epoch number: 81 with loss: 0.6310698390007019
Epoch number: 91 with loss: 0.6276214718818665
Epoch number: 101 with loss: 0.6240186095237732
Epoch number: 111 with loss: 0.6202120184898376
Epoch number: 121 with loss: 0.6161925792694092
Epoch number: 131 with loss: 0.6119955778121948
Epoch number: 141 with loss: 0.6075876951217651
Epoch number: 151 with loss: 0.602949857711792
Epoch number: 161 with loss: 0.5981548428535461
Epoch number: 171 with loss: 0.5932035446166992
Epoch number: 181 with loss: 0.5881384611129761
Epoch number: 191 with loss: 0.5829936265945435
Epoch number: 201 with loss: 0.577807605266571
Epoch

In [259]:
#predictions
y_pred = []

with torch.no_grad():
    for i, data in enumerate(X_test):
        prediction = ann(data)
        y_pred.append(prediction.argmax()) 

In [260]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.85      0.82       150
           1       0.68      0.62      0.65        81

    accuracy                           0.77       231
   macro avg       0.74      0.73      0.74       231
weighted avg       0.76      0.77      0.76       231



Among all the Adam Optimizers i found the best results with SGD, stochastic gradient descent (optionally with momentum), also i tried changing the loss function but that doesn't work for me.
The new optimizer-SDG(stochastic gradient descent) performed relatively better, when compared to Adam in class this may be because
of random selection of the sample for each iteration instead of sum all the samples in the dataset. 
From the classification_report the precision and recall values are 68 and 62 respectively, which are not bad when compared to Adam optimizer.


2. Write a function that lists and counts the number of divisors for an input value.
Example 1:
    
Input: 5
    
Output: “There are 2 divisors: 1 and 5”
Example 2:
    
Input: 40
    
Output: “There are 8 divisors: 1, 2, 4, 5, 8, 10, 20, and 40"

In [129]:
# define function divisor_count that counts the number of divisors for the input value
def divisor_count (x):
    divisors = []
    if x == 1:
        print(f'There is 1 divisor: 1')
    for i in range(1, x+1):
        if x%i==0:
            divisors.append(i)
    print(f'There are {len(divisors)} divisor : {str((divisors)[:-1])[1:-1]} and {divisors[-1]}')

In [130]:
divisor_count(5)

There are 2 divisor : 1 and 5


In [131]:
divisor_count(40)

There are 8 divisor : 1, 2, 4, 5, 8, 10, 20 and 40
